In [12]:
import pandas as pd
import numpy as np
energy = pd.read_csv('Energy Indicators.csv',skiprows=16,skipfooter=38,engine='python')
energy.drop(energy.columns[[0,1]], axis=1,inplace=True)
energy=energy.dropna(axis=1,how='all')

for col in energy.columns:
    if col[:]=='Energy Supply per capita':
        energy.rename(columns={col:'Energy Supply per Capita'},inplace=True)
    if col[:]=='Renewable Electricity Production':
        energy.rename(columns={col:'% Renewable'},inplace=True)
    energy.rename(columns={ energy.columns[0]: "Country" },inplace=True)
    
energy.dropna(subset=['Country'], how = 'all',inplace=True)

#replace empty values with Nan
energy[['Energy Supply','Energy Supply per Capita','% Renewable']] = energy[['Energy Supply','Energy Supply per Capita','% Renewable']].apply(pd.to_numeric,errors='coerce')      
energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x:x*1000000)

energy['Country'].replace(
    to_replace=['United States of America20','United States Virgin Islands','Bolivia (Plurinational State of)',
               'China2','China, Hong Kong Special Administrative Region3','China, Macao Special Administrative Region4',
               'Denmark5','France6','Greenland7','Indonesia8','Iran (Islamic Republic of)','Italy9',
               'Japan10','Kuwait11','Micronesia (Federated States of)','Netherlands12','Portugal13','Saudi Arabia14',
               'Serbia15','Ukraine18','Venezuela (Bolivarian Republic of)','United Kingdom of Great Britain and Northern Ireland19'],
    value=['United States','United Virgin','Bolivia','China','Hong Kong','China, Macao Special Administrative Region',
          'Denmark','France','Greenland','Indonesia','Iran','Italy','Japan','Kuwait','Micronesia','Netherlands',
          'Portugal','Saudi Arabia','Serbia','Ukraine','Venezuela','United Kingdom'],
    inplace=True
)


GDP = pd.read_csv('world_bank.csv',skiprows=4)
GDP['Country Name'].replace(
    to_replace=['Korea, Rep.','Iran, Islamic Rep.','Hong Kong SAR, China'],
    value=['South Korea','Iran','Hong Kong'],
    inplace=True
)
GDP.dropna(axis=1,how='all',inplace=True)

ScimEn = pd.read_csv("scimagojr.csv")

energy = energy.set_index('Country')
GDP = GDP.set_index('Country Name')
ScimEn = ScimEn.set_index('Country')

intermediate_merge = pd.merge(energy,  GDP , how='outer', left_index=True, right_index=True)
final_merge = pd.merge(intermediate_merge,  ScimEn, how = 'outer' , left_index=True, right_index=True).sort_values(['Rank'],ascending=True)

for i in range(1960,2005):
    final_merge.drop(str(i),axis=1,inplace=True)
    
final_merge.head(15)

cols = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
final_merge = final_merge[cols]

result_df = final_merge.head(15)

In [13]:
result_df.columns

Index([u'Rank', u'Documents', u'Citable documents', u'Citations',
       u'Self-citations', u'Citations per document', u'H index',
       u'Energy Supply', u'Energy Supply per Capita', u'% Renewable', u'2006',
       u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013', u'2014',
       u'2015'],
      dtype='object')

In [3]:
def answer_three():
    Top15 = final_merge.head(15)
   
    return Top15[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015']].mean(axis=1).sort_values(ascending=False)

In [4]:
answer_three()


United States         1.562297e+13
China                 6.940046e+12
Japan                 5.176323e+12
Germany               3.532694e+12
United Kingdom        2.724782e+12
France                2.688773e+12
Italy                 2.130288e+12
Brazil                1.989028e+12
Russian Federation    1.656010e+12
Canada                1.611540e+12
India                 1.594349e+12
Spain                 1.406409e+12
Australia             1.202763e+12
South Korea           1.165810e+12
Iran                  4.369708e+11
dtype: float64

In [10]:
def answer_four():
    Top15 = final_merge.head(15)
    avgGDP = answer_three()
    avgGDP = avgGDP.to_frame().rename(columns= {0: 'AvgGDP'})
    final_df = pd.merge(GDP,avgGDP,how="outer",left_index=True, right_index=True).sort_values(['AvgGDP'],ascending=False)
    final_df = final_df[5:6]    
    return final_df['2016'].values[0] - final_df['2006'].values[0]
    

In [11]:
answer_four()

140442057078.75

In [14]:
result_df[['Energy Supply per Capita']].mean().values[0]

150.5

In [21]:
def answer_six():
    Top15 = final_merge.head(15)
    df = Top15.loc[Top15['% Renewable'] ==  Top15[['% Renewable']].max().values[0]]
    df = pd.DataFrame(df['% Renewable'])   # convert to dataframe as we need tuples as return value
    return tuple(df.to_records().tolist())

In [20]:
answer_six()

(('Brazil', 69.65),)